## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [ ]:
from __future__ import print_function
import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

### Load imdb Data

In [ ]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [ ]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
# tf.keras.datasets.imdb.load_data(
#     path='imdb.npz',
#     num_words=None,
#     skip_top=0,
#     maxlen=None,
#     seed=113,
#     start_char=1,
#     oov_char=2,
#     index_from=3,
#     **kwargs)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

### Decode data into Sentence

In [ ]:
word_index = imdb.get_word_index()
word_index["<PAD>"] = -3
word_index["<START>"] = -2
word_index["<UNK>"] = -1
word_index["<UNUSED>"] = 0
# Offset the indices by 3 because of the value of starting index_from
# and imdb.get_word_index() dictionary begins at 1
inv_word_index = {v+3: k for k, v in word_index.items()}

def decode_review(encoded_review):
    decoded_review = " ".join([inv_word_index.get(i, "?") for i in encoded_review])
    return decoded_review

In [ ]:
print(decode_review(X_train[0]))
print(y_train[0])
print(decode_review(X_train[1]))
print(y_train[1])
print(decode_review(X_train[2]))
print(y_train[2])

### Pad (or Truncate) Sequences

In [ ]:
sequence = [[1], [2, 3], [4, 5, 6]]
pad_sequences(sequence, maxlen=2, padding='pre', truncating='post')

In [ ]:
# This pads (or truncates) the sequences so that they are of the maximum length
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
print(X_train[0, :])  #Here's what an example sequence looks like
print(decode_review(X_train[0]))

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [ ]:
## Let's build a RNN
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  # This layer takes each integer in the sequence and 
                                                            # embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                        kernel_initializer=initializers.RandomNormal(stddev=0.001),
                        recurrent_initializer=initializers.Identity(gain=1.0),
                        activation='relu',
                        input_shape=X_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=0.0001)

model_rnn.compile(loss='binary_crossentropy',
                  optimizer=rmsprop,
                  metrics=['accuracy'])

In [ ]:
model_rnn.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(X_test, y_test))

In [ ]:
score, acc = model_rnn.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

## Exercise
### Your Turn

Now do it yourself:
- Prepare the data to use sequences of length 80 rather than length 30.  Did it improve the performance?
- Try different values of the "max_features".  Can you improve the performance?
- Try smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?